In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time
import os

In [ ]:
def obtain_data(day,month,year):
    '''Function that reads in the data from the ipu archives'''
    df = pd.read_html('http://archive.ipu.org/wmn-e/arc/classif{}{}{}.htm'.format(day,month,year), header=[1])[1]
    return df

In [ ]:
def clean(df):
    '''Function that corrects the column headers'''
    # sets the multi-level columns using the columns and first row
    df.columns=pd.MultiIndex.from_tuples(zip(df.columns,df.iloc[0].values))
    # removes the first row
    df = df.iloc[1:, :]
    
    return df

In [ ]:
# extract  all links from ipu archive page
links = []
url = "http://archive.ipu.org/wmn-e/classif-arc.htm" # url to ipu archives page
page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

for link in soup.find_all('a'):
    links.append(link.get('href'))
    
# extract dates from links  
dates = [re.search(r'classif(.*?).htm', link).group(1) for link in links if re.search(r'classif(.*?).htm', link) is not None and len(re.search(r'classif(.*?).htm', link).group(1)) ==6]
dates = [(date[0:2],date[2:4],date[4:6]) for date in dates]

In [ ]:
# loop through months and years 1997-2019 and pull the data, save each file to the data/world_data folder
for d,m,y in dates:
    try:
        df = clean(obtain_data(d,m,y))
        # save dataframe to folder but change format of name to make formatting consistent (ex: 'wd_month_year')
        if (m[0]=='0') & (int(y)>19):
            df.to_csv('data/world_data/' + 'wd_' + m[1] + '_' + '19' + y + '.csv', index=False)
        elif (m[0]=='0') & (int(y)<=19):
            df.to_csv('data/world_data/' + 'wd_' + m[1] + '_' + '20' + y + '.csv', index=False)
        elif (m[0]>'0') & (int(y)>19):
            df.to_csv('data/world_data/' + 'wd_' + m + '_' + '19' + y + '.csv', index=False)
        else:
            df.to_csv('data/world_data/' + 'wd_' + m + '_' + '20' + y + '.csv', index=False)
    except:
        print(str(m) + ',' + y + ' did not work.' )
    time.sleep(15)